In [ ]:
!pip install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 691 kB 5.3 MB/s 
     |████████████████████████████████| 240 kB 48.7 MB/s 
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=dbb58863e01f9cd915b8a2295d40c2f54db90ac3587b18835442ca44aa8886d5
  Stored in directory: /root/.cache/pip/wheels/f3/e3/f2/1de1c2e3ed742e1df73e0f15d58864e50c7e64f607b548d6cf
Successfully built emoji


In [ ]:
import glob
import stanza
import csv
import pandas as pd
from google.colab import drive
from google.colab import files

In [ ]:
# connect to Google Drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# cd into the directory where the data is
%cd /content/gdrive/MyDrive/Colab Notebooks/data/school_files

/content/gdrive/MyDrive/Colab Notebooks/data/school_files


In [ ]:
# load pipeline
nlp = stanza.Pipeline(lang='pt')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: pt (Portuguese):
| Processor    | Package |
--------------------------
| tokenize     | bosque  |
| mwt          | bosque  |
| pos          | bosque  |
| lemma        | bosque  |
| depparse     | bosque  |
| constituency | cintil  |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: constituency
INFO:stanza:Done loading processors!


In [ ]:
enclitic_forms = ["-me", "-te", "-se", "-si", "-vos", "-lhe", "-o", "-lo", "-no", "-a", "-la", "-na"]
proclitic_forms = ["me", "te", "se", "si", "vos", "lhe", "lhes", "o", "os", "no", "nos", "a", "na", "nas"]
mesoclitic_forms = ["-ei", "-ás", "-á", "-emos", "-eis", "-ão", "-ia", "-íamos", "-iam"]
punctuation = "!*()\"+<>?:.,;"

In [ ]:
# create functions for clitic extraction
def count_enclitics(text):
  enclisis = 0
  token_list=[]
  a=''
  for c in text:
    if c in punctuation:
      a = text.replace(c, c+" ")
      a = text.replace(c, "")
  token_list=a.split(" ")
  for token in token_list:
    if '-' in token:
      hyphen=token.count("-")
      if hyphen == 1:
        for pron in enclitic_forms:
          if pron in token:
            expanded = token.replace('-', ' ')
            verb_check = nlp(expanded)
            for sent in verb_check.sentences:
                for word in sent.words:
                  if word.upos == 'VERB':
                    enclisis+=1
  return enclisis                 
    
def count_mesoclitics(text):    
  mesoclisis = 0
  a=''
  token_list=[]
  for c in text:
    if c in punctuation:
      a = text.replace(c, c+" ")
      a = text.replace(c, "")
  token_list= a.split(" ")
  for token in token_list:
    if '-' in token:
      hyphen=token.count("-")
      if hyphen==2:
        for meso_pron in mesoclitic_forms:
          if token.endswith(meso_pron):
            mesoclisis+=1
  return mesoclisis

def count_proclitics(text):  
  pos_dep = ""
  c = nlp(text)
  for sentence in c.sentences:
    for word in sentence.words:
    # for pron in proclitic_forms:
      # if word.text == pron:
      pos_dep+=word.pos
      pos_dep+=word.deprel        
  proclisis = pos_dep.count('PRONobjVERBroot')
  proclisis = proclisis + pos_dep.count('PRONiobjVERBroot')
  return proclisis

In [ ]:
# call functions and write results into Google Drive files
data = glob.glob(f"/content/gdrive/MyDrive/Colab Notebooks/data/school_files/*.txt")
filenames=[]
clitics=[]
enclitics=[]
mesoclitics=[]
proclitics=[]
for file in data:  
  text = open(file, "r")
  text=text.read().replace('\n', '')
  text=str(text)
  filenames.append(file.replace("/content/gdrive/MyDrive/Colab Notebooks/data/school_files/", ""))
  en = count_enclitics(text)
  enclitics.append(en)
  mes = count_mesoclitics(text)
  mesoclitics.append(mes)
  pro = count_proclitics(text)
  proclitics.append(pro)
  total = en + mes + pro
  clitics.append(total)

list_of_tuples = list(zip(filenames, enclitics, mesoclitics, proclitics, clitics))  
df = pd.DataFrame(list_of_tuples, columns=['filenames', 'enclitics', 'mesoclitics', 'proclitics', 'clitics'])

In [ ]:
df.head(15)

In [ ]:
# Open dataframe with CTAP features
features = pd.read_csv("School_levels_all_features.csv", encoding= 'unicode_escape')

In [ ]:
# Check order of files
features.head(15)

In [ ]:
# Delete filenames from clitics dataframe
df = df.drop('filenames', axis=1)

In [ ]:
# Merge clitic features into main dataframe
frames = features, df
df = pd.concat(frames)

In [ ]:
# Add clitics-per-VP features
df['Syntactic Complexity: Clitics per VP'] = df['Number of Clititcs'] / df['Number of Syntactic Constituents: Verb Phrase']

In [ ]:
df['Syntactic Complexity: Mesoclitics per VP'] = df['Number of Mesoclitics'] / df['Number of Syntactic Constituents: Verb Phrase']

In [ ]:
df['Syntactic Complexity: Enclitics per VP'] = df['Number of Enclitics'] / df['Number of Syntactic Constituents: Verb Phrase']

In [ ]:
df['Syntactic Complexity: Proclitics per VP'] = df['Number of Proclitics'] / df['Number of Syntactic Constituents: Verb Phrase']

In [ ]:
# Save new formatted data frame
df.to_csv('School_levels_all_features.csv', encoding='utf-8-sig') 
files.download('School_levels_all_features.csv')